In [1]:
%autosave 0

Autosave disabled


In [2]:
import joblib
import pandas as pd
import numpy as np
import lightgbm as lgb

In [3]:
pd.set_option("display.max_rows", 18)

In [4]:
# 32日分のデータでテストする
TEST_DAYS = 32

In [5]:
#xtrain, ytrain, harai = joblib.load("../lgbm_today_20240401/20221228.pkl")

In [6]:
#ytrain = xtrain["jyuni"].copy()

In [7]:
#xtrain = xtrain.iloc[:, :226].drop(["ijyo", "jyuni"], axis=1).copy()

In [8]:
#ytrain

In [9]:
#joblib.dump((xtrain, ytrain, harai), "data.pkl")

# Read Data

In [10]:
xall, yall, harai = joblib.load("./data.pkl")

In [11]:
# 説明変数
# ある程度前処理済 詳しくはJV-Data490.xlsxのコード表を参照
# _1は1走前のデータ、_5は5走前のデータを表す
xall

,raceid,horseid,racedate,futan,umaban,wakuban,blinker,age,bataijyu,zogen,...,grade_5,wintime_5,lap_s3_5,lap_s4_5,lap_l3_5,lap_l4_5,f,ff,m,mf
0,2020010506010101,2017103291,20200105,54.0,1,1,0,3.0,432.0,4.0,...,9,NaN,NaN,NaN,NaN,NaN,1.120233e+07,1.120209e+07,1.225691e+07,1.120196e+07
1,2020010506010101,2017101861,20200105,54.0,2,1,0,3.0,424.0,-4.0,...,9,NaN,NaN,NaN,NaN,NaN,1.120218e+07,1.140407e+07,1.226263e+07,1.120219e+07
2,2020010506010101,2017103186,20200105,54.0,3,2,0,3.0,458.0,0.0,...,9,NaN,NaN,NaN,NaN,NaN,1.120225e+07,1.120200e+07,1.226112e+07,1.140592e+07
3,2020010506010101,2017102095,20200105,53.0,4,2,0,3.0,464.0,10.0,...,9,NaN,NaN,NaN,NaN,NaN,1.120230e+07,1.120201e+07,1.226203e+07,1.120212e+07
4,2020010506010101,2017103287,20200105,52.0,5,3,1,3.0,460.0,-2.0,...,9,NaN,NaN,NaN,NaN,NaN,1.120226e+07,1.140640e+07,1.226179e+07,1.140456e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143318,2022122809060912,2018105223,20221228,53.0,12,6,0,4.0,452.0,0.0,...,4,69.4,33.6,45.3,35.8,46.9,1.140007e+09,1.140005e+09,1.220065e+09,1.140006e+09
143319,2022122809060912,2017106137,20221228,56.0,13,7,0,5.0,492.0,6.0,...,4,67.3,32.8,44.1,34.5,45.4,1.120002e+09,1.140004e+09,1.220050e+09,1.120002e+09
143320,2022122809060912,2017100461,20221228,56.0,14,7,0,5.0,504.0,4.0,...,4,80.4,35.6,47.1,33.3,44.8,1.120002e+09,1.140004e+09,1.220055e+09,1.120002e+09
143321,2022122809060912,2018106389,20221228,52.0,15,8,0,4.0,434.0,2.0,...,4,68.4,33.7,45.0,34.7,45.7,1.120002e+09,1.140005e+09,1.220063e+09,1.120002e+09


In [12]:
# fは父の血統コード
# 7桁と9桁のコードが混在しているので後日修正
xall["f"].fillna(0).astype(int)

0           11202326
1           11202183
2           11202251
3           11202295
4           11202261
             ...    
143318    1140006549
143319    1120002238
143320    1120001906
143321    1120002293
143322    1120002352
Name: f, Length: 143323, dtype: int64

In [13]:
# 目的変数（着順）
# 競争中止の場合等に0になる
yall

0          2
1         12
2          3
3         11
4          8
          ..
143318    12
143319    16
143320    11
143321     7
143322     8
Name: jyuni, Length: 143323, dtype: int64

In [14]:
# 払戻金
harai

,RaceID,TorokuTosu,SyussoTosu,FuseirituFlag,TokubaraiFlag,HenkanFlag,HenkanUma,HenkanWaku,HenkanDoWaku,PayTansyo,...,PayReserved1,PayUmatan,PaySanrenpuku,PaySanrentan,Year,MonthDay,JyoCD,Kaiji,Nichiji,RaceNum
0,2020010606010203,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '09', 'Pay': '000000200', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0913', 'Pay': '000000510', 'Ninki':...","[{'Kumi': '080913', 'Pay': '000001270', 'Ninki...","[{'Kumi': '091308', 'Pay': '000003840', 'Ninki...",2020,0106,06,01,02,03
1,2022010507010112,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '13', 'Pay': '000001530', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '1315', 'Pay': '000004070', 'Ninki':...","[{'Kumi': '061315', 'Pay': '000002720', 'Ninki...","[{'Kumi': '131506', 'Pay': '000026590', 'Ninki...",2022,0105,07,01,01,12
2,2020010506010101,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '07', 'Pay': '000000360', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0701', 'Pay': '000003650', 'Ninki':...","[{'Kumi': '010307', 'Pay': '000098210', 'Ninki...","[{'Kumi': '070103', 'Pay': '000280650', 'Ninki...",2020,0105,06,01,01,01
3,2020010606010204,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '12', 'Pay': '000000280', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '1211', 'Pay': '000003040', 'Ninki':...","[{'Kumi': '031112', 'Pay': '000000920', 'Ninki...","[{'Kumi': '121103', 'Pay': '000008340', 'Ninki...",2020,0106,06,01,02,04
4,2020010606010205,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '07', 'Pay': '000000380', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0716', 'Pay': '000001730', 'Ninki':...","[{'Kumi': '070916', 'Pay': '000002130', 'Ninki...","[{'Kumi': '071609', 'Pay': '000009690', 'Ninki...",2020,0106,06,01,02,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10363,2022122806050903,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '01', 'Pay': '000000600', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0107', 'Pay': '000004600', 'Ninki':...","[{'Kumi': '010307', 'Pay': '000023600', 'Ninki...","[{'Kumi': '010703', 'Pay': '000094870', 'Ninki...",2022,1228,06,05,09,03
10364,2022122806050902,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '04', 'Pay': '000000750', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0406', 'Pay': '000030980', 'Ninki':...","[{'Kumi': '040611', 'Pay': '000013110', 'Ninki...","[{'Kumi': '040611', 'Pay': '000138570', 'Ninki...",2022,1228,06,05,09,02
10365,2022122809060911,13,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]

# Preprocess

In [15]:
# コードが文字列になっているので数値に変更
for col in xall.columns:
    if xall[col].dtype=="object":
        xall[col] = xall[col].astype(float)
        print(col)

KisyuCode
ChokyosiCode
BanusiCode
BreederCode
KisyuCode_1
KisyuCode_2
KisyuCode_3
KisyuCode_4
KisyuCode_5


In [16]:
# 2値分類にする
yall_binary = yall.apply(lambda x: 1 if x==1 else 0)

# Train Test Split

In [17]:
# racedateでデータを分割
rd = xall["racedate"].unique()[-TEST_DAYS] 

# split data
xtrain = xall[xall['racedate']<rd].reset_index(drop=True).copy()
ytrain = yall_binary[xtrain.index].reset_index(drop=True).copy()
xtest = xall[xall['racedate']>=rd].reset_index(drop=True).copy()
ytest = yall_binary[xtest.index].reset_index(drop=True).copy()

In [18]:
print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

(129888, 224) (129888,) (13435, 224) (13435,)


# Train

In [19]:
# parameters
num_boost_round = 100
early_stopping_round = 10
parameters = {
 'max_depth': 10,
 'min_data_in_leaf': 50,
 'learning_rate': 0.01,
 'seed': 1,
 'objective': 'binary',
 'metric': 'binary_logloss',
}

In [20]:
# validation dataを設定する（ここではtraining_data=validation_dataとしている）
xtrain_val = xtrain.copy()
ytrain_val = ytrain.copy()
xtest_val = xtrain.copy()
ytest_val = ytrain.copy()

In [21]:
# LightGBM用の設定
train_data = lgb.Dataset(xtrain_val, label=ytrain_val) 
valid_data = lgb.Dataset(xtest_val, label=ytest_val)

In [22]:
gbm = lgb.train(parameters, 
                train_data,
                valid_sets=[valid_data],
                num_boost_round=num_boost_round,
                callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_round, 
                            verbose=1)]
               )

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 9405, number of negative: 120483
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.211140 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17586
[LightGBM] [Info] Number of data points in the train set: 129888, number of used features: 224
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.072409 -> initscore=-2.550267
[LightGBM] [Info] Start training from score -2.550267
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[

In [23]:
# 特徴量の重要度の表示
importance = pd.DataFrame(gbm.feature_importance(), index=xtrain.columns, columns=['importance'])
#importance.sort_values("importance", ascending=False)
for k, v in importance.sort_values("importance", ascending=False).iterrows():
    print(v.name, v.importance)

odds 865
f 169
cls 169
m 139
ff 122
horseid 107
goal_1 55
honsyokin_1 46
3c_1 39
odds_5 36
KisyuCode 33
mf 33
4c_5 29
odds_2 28
odds_1 27
honsyokin_3 24
l3_1 24
3c_5 24
KisyuCode_3 23
BanusiCode 23
ninki_1 22
raceid 22
umaban 22
lap_s3_2 20
lap_s3_5 19
KisyuCode_5 18
4c_1 17
nyusen_1 17
year_5 17
l3_4 17
diff_1 16
lap_s4_5 16
goal_3 15
odds_3 15
jyuryo 14
time_5 14
zogen 14
lap_l4_4 14
KisyuCode_2 13
syubetu 13
diff_2 13
ChokyosiCode 13
umaban_3 13
3c_4 13
wintime_5 12
lap_l4_2 12
l3_2 12
lap_s4_4 11
lap_s3_4 11
lap_l3_4 11
futan_2 11
lap_s4_3 11
bataijyu 11
bataijyu_4 11
KisyuCode_1 11
3c_2 10
honsyokin_2 10
diff_3 10
jyuryo_2 10
bataijyu_1 10
goal_4 10
4c_2 10
time_3 9
honsyokin_5 9
nyusen_5 9
umaban_5 9
grade_2 9
zogen_2 8
zogen_4 8
odds_4 8
lap_s3_3 8
lap_l4_3 8
lap_s4_1 8
lap_s3_1 8
honsyokin_4 8
wintime_3 8
wakuban_1 8
BreederCode 8
ninki_4 7
lap_s4_2 7
time_2 7
time_4 7
goal_2 7
jyo_2 6
wakuban_3 6
ninki_3 6
year_3 6
tosu_4 6
zogen_1 6
jyo_4 6
time_1 6
wintime_4 6
4c_4 5
lap_l4_

# Predict

In [24]:
ypred = gbm.predict(xtest)

In [25]:
ypred

array([0.029249  , 0.07859686, 0.029249  , ..., 0.05534488, 0.03176091,
       0.06358456])

In [26]:
# logloss score
from sklearn.metrics import log_loss
log_loss(ytest.values, ypred)

0.2825043216329923

# Evaluate

In [27]:
def calc_mrr(ypred, ytrue):
    '''
    Args:
        ypred（numpy.array): 予想順位（1以上） [n, MAX_HORSE]
        ytrue（numpy.array): 確定順位 [n, MAX_HORSE]
    
    Returns:
        mrr (int)
    '''
    res = []
    for i in range(ytrue.shape[0]):
        #1着馬を何位と予想しているか 同着があるので小さい方を取る
        indices = np.where(ytrue[i, :] == 1) 
        if indices[0].size > 0: # 発走前のレースで1着がない場合がある
            res.append(1 / ypred[i, np.min(indices)])
    return np.mean(res)

In [28]:
def get_reports(df_res):
    """概要

    単勝の予想結果をレポートとして出力する。

    Args:
        df_res (pd.DataFrame): get_win_resultsから出力された結果集計データ

    Returns:
        None
        
    Note:
        返還があるレースは集計対象外
    """

    #的中率
    tekichu = df_res.apply(lambda x: sum([y in x["true"] for y in x["pred"]])>=1, axis=1).astype(int).values
    print("レース数", df_res.shape[0])
    print("的中数", np.sum(tekichu))
    print("的中率", np.mean(tekichu), np.std(tekichu, ddof=1), np.std(tekichu, ddof=1) / np.sqrt(df_res.shape[0]))

    #回収率
    modoshi = df_res.apply(lambda x: sum([x["pay"][y] if y in x["true"] else 0 for y in x["pred"]]) + x["henkan"]*100, axis=1).values
    harai = df_res.apply(lambda x: len(x["pred"]), axis=1).values * 100
    print("払戻金額", np.sum(modoshi))
    print("回収率", np.mean(modoshi/harai), np.std(modoshi/harai, ddof=1), np.std(modoshi/harai, ddof=1) / np.sqrt(df_res.shape[0]))
    print("※平均・標準偏差・標準誤差")


In [29]:
def get_win_results(df_bet, df_kekka):
    """概要

    単勝の買い目を入力し、払戻データとマージして出力する。

    Args:
        df_bet (pd.DataFrame): raceid, winを含むデータ
        df_kekka (pd.DataFrame): JRA-VANから抽出した払戻データ

    Returns:
        df_results (pd.DataFrame): レース毎に集計したデータ
        
    Note:
        同着があるのでtrueはlist、payはdictになっている
    """

    #raceidごとに集計
    res = []
    for raceid in df_bet["raceid"].tolist():
        #predict
        pred = df_bet[df_bet["raceid"]==raceid]["win"].values[0] #list
        #check win or loss
        try:
            tmp = df_kekka[df_kekka.raceid==raceid].iloc[0].to_dict() #target race
        except:
            print("払戻データに対象レースが存在しません。", raceid) 
            continue

        true = [x["Umaban"] for x in tmp["PayTansyo"] if x["Umaban"]!='  '] #同着があるのでtrueとpayはリスト
        pay = {x["Umaban"]:int(x["Pay"]) for x in tmp["PayTansyo"] if x["Umaban"]!='  '} 
        henkan = sum([int(tmp["HenkanUma"][x-1]) for x in pred])  #返還
        pred = [f"{x:02}" for x in pred]  # to str  
        res.append({"pred":pred, "true":true, "pay":pay, "henkan":henkan, "raceid":raceid})     
        
    df_res = pd.DataFrame(res)
    get_reports(df_res)

    return df_res


In [30]:
def evaluate(xtest, ytest, ypred, harai):
      
    #jyuni レース数×18の行列にそれぞれの順位を格納する
    tmp = xtest.copy()
    tmp["jyuni"] = ytest
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
    jyuni = np.array([[tmp[i][j] if j<len(tmp[i]) else j+1 for j in range(18)] for i in range(tmp.shape[0])]) #padding
    print("jyuni\n", jyuni)

    #pred レース数×18の行列にそれぞれの予想順位を格納する（スコアの大きい順）
    tmp = xtest.copy()
    tmp["pred"] = ypred
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: np.argsort(np.argsort(-x["pred"].values)) + 1)
    pred = np.array([[tmp[i][j] if j<len(tmp[i]) else j+1 for j in range(18)] for i in range(tmp.shape[0])]) #padding
    print("pred\n", pred)

    #mrr
    mrr = calc_mrr(pred, jyuni)
    print("mrr\n", mrr)
        
    #df_bet
    df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0]) #ユニークなraceidを抽出しているだけ

    #win
    kaime = [[np.where(pred[i, :]==1)[0][0]+1] for i in range(pred.shape[0])] # スコア1位の単勝を買う #don't forget +1
    df_bet["win"] = kaime

    harai = harai.copy()
    harai.columns = [x.replace("RaceID", "raceid") for x in harai.columns]
    harai.raceid = harai.raceid.astype(int)

    #結果の表示
    df_res = get_win_results(df_bet, harai)

    return df_bet, df_res

In [31]:
df_bet, df_res = evaluate(xtest, yall[xtest.index].reset_index(drop=True).copy(), ypred, harai)

jyuni
 [[ 2 12  3 ... 16 17 18]
 [ 6  7 13 ... 16 17 18]
 [11  7  6 ... 16 17 18]
 ...
 [ 9  3 12 ... 16 17 18]
 [10 13 11 ... 16 17 18]
 [ 6 10  9 ...  3 17 18]]
pred
 [[10  4 11 ... 16 17 18]
 [10 12  3 ... 16 17 18]
 [10  8  5 ... 16 17 18]
 ...
 [ 3  4  5 ... 16 17 18]
 [ 5  8  1 ... 16 17 18]
 [ 3  1  5 ...  7 17 18]]
mrr
 0.23057792917401268
レース数 984
的中数 345
的中率 0.35060975609756095 0.47740357678115486 0.015219070164440487
払戻金額 84130
回収率 0.854979674796748 1.2649012521192773 0.04032357913379549
※平均・標準偏差・標準誤差


In [32]:
# 予想
df_bet

,raceid,win
0,2022091806040401,[4]
1,2022091806040402,[13]
2,2022091806040403,[5]
3,2022091806040404,[14]
4,2022091806040405,[1]
...,...,...
979,2022122809060908,[10]
980,2022122809060909,[16]
981,2022122809060910,[11]
982,2022122809060911,[3]


In [33]:
# 予想と結果
df_res

,pred,true,pay,henkan,raceid
0,[04],[04],{'04': 270},0,2022091806040401
1,[13],[10],{'10': 1540},0,2022091806040402
2,[05],[05],{'05': 280},0,2022091806040403
3,[14],[14],{'14': 350},0,2022091806040404
4,[01],[14],{'14': 420},0,2022091806040405
...,...,...,...,...,...
979,[10],[10],{'10': 290},0,2022122809060908
980,[16],[16],{'16': 410},0,2022122809060909
981,[11],[13],{'13': 2820},0,2022122809060910
982,[03],[04],{'04': 520},0,2022122809060911


In [34]:
# 的中レース
df_res[df_res.pred==df_res.true]

,pred,true,pay,henkan,raceid
0,[04],[04],{'04': 270},0,2022091806040401
2,[05],[05],{'05': 280},0,2022091806040403
3,[14],[14],{'14': 350},0,2022091806040404
7,[07],[07],{'07': 180},0,2022091806040408
12,[02],[02],{'02': 190},0,2022091807050401
...,...,...,...,...,...
971,[06],[06],{'06': 230},0,2022122806050912
975,[03],[03],{'03': 190},0,2022122809060904
977,[07],[07],{'07': 150},0,2022122809060906
979,[10],[10],{'10': 290},0,2022122809060908


In [35]:
xtest["pred"] = ypred
xtest["jyuni"] = yall[xtest.index].reset_index(drop=True).copy()

In [36]:
# 2022年12月28日 6回 阪神 9日目 12レース
xtest[xtest["raceid"]==2022122809060912]

,raceid,horseid,racedate,futan,umaban,wakuban,blinker,age,bataijyu,zogen,...,lap_s3_5,lap_s4_5,lap_l3_5,lap_l4_5,f,ff,m,mf,pred,jyuni
13419,2022122809060912,2019105237,20221228,53.0,1,1,0,3.0,422.0,8.0,...,33.6,44.8,34.3,45.4,1.120002e+09,1.120002e+09,1.220054e+09,1.140004e+09,0.100821,6
13420,2022122809060912,2018106055,20221228,55.0,2,1,0,4.0,486.0,6.0,...,34.0,45.5,35.1,46.6,1.120002e+09,1.120002e+09,1.220065e+09,1.120002e+09,0.170365,10
13421,2022122809060912,2018106077,20221228,57.0,3,2,1,4.0,492.0,-4.0,...,33.6,45.5,35.5,46.8,1.120002e+09,1.120002e+09,1.220064e+09,1.120002e+09,0.076115,9
13422,2022122809060912,2018103926,20221228,55.0,4,2,1,4.0,468.0,-2.0,...,34.3,45.6,34.6,45.9,1.120002e+09,1.120001e+09,1.220063e+09,1.140006e+09,0.083841,15
13423,2022122809060912,2018104949,20221228,54.0,5,3,0,4.0,474.0,12.0,...,33.7,45.9,36.5,47.9,1.120002e+09,1.120002e+09,1.220065e+09,1.120002e+09,0.039434,3
13424,2022122809060912,2017102603,20221228,54.0,6,3,1,5.0,484.0,4.0,...,33.9,45.2,34.5,45.8,1.120002e+09,1.120002e+09,1.220054e+09,1.120002e+09,0.036537,11
13425,2022122809060912,2019102979,20221228,56.0,7,4,0,3.0,498.0,4.0,...,33.6,45.0,34.9,46.0,1.120002e+09,1.120002e+09,1.220058e+09,1.140003e+09,0.116529,1
13426,2022122809060912,2016110041,20221228,52.0,8,4,0,6.0,452.0,2.0,...,33.7,45.9,36.5,47.9,1.140655e+07,1.140460e+07,1.243019e+07,1.140662e+07,0.029249,6
13427,2022122809060912,2018100740,20221228,52.0,9,5,0,4.0,476.0,32.0,...,34.3,45.9,34.7,46.1,1.120240e+07,1.120209e+07,1.226462e+07,1.120213e+07,0.029249,12
13428,2022122809060912,2017104873,20221228,56.0,10,5,0,5.0,508.0,4.0,...,30.9,43.6,36.6,48.8,1.120002e+09,1.120002e+09,1.220058e+09,1.120002e+09,0.054577,2
